In [8]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re


In [9]:
! pip install nltk

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
data = pd.read_csv('/content/Spam Email Detection - spam.csv')

In [12]:
data.dropna(inplace=True)

In [13]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    # Stemming
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    return ' '.join(words)

In [15]:
data['clean_text'] = data['v2'].apply(preprocess_text) # replace 'v2' with the actual column name from your data

In [17]:
# Assuming the column containing spam/ham information is named 'v1'
# Replace 'v1' with the actual column name if different

data = pd.read_csv('/content/Spam Email Detection - spam.csv')
data.dropna(inplace=True)

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    # Stemming
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    return ' '.join(words)

data['clean_text'] = data['v2'].apply(preprocess_text) # replace 'v2' with the actual column name from your data

# Assuming the column containing spam/ham information is named 'v1'
# Replace 'v1' with the actual column name if different
data['label'] = data['v1'].map({'spam': 1, 'ham': 0})

# Print the columns of the DataFrame to verify
# and adjust the column name in the code accordingly.
print(data.columns)

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'clean_text',
       'label'],
      dtype='object')


In [18]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['clean_text']).toarray()
y = data['label']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [21]:
y_pred = model.predict(X_test)

In [22]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2


Confusion Matrix:
 [[2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [23]:
def predict_email(email):
    processed_email = preprocess_text(email)
    vectorized_email = tfidf.transform([processed_email]).toarray()
    prediction = model.predict(vectorized_email)[0]
    return "Spam" if prediction == 1 else "Ham"

In [26]:
# Test example
test_email = "Congratulations! You've won a $1,000 Walmart gift card. Click here to claim your prize."
print("\nTest Email Prediction:", predict_email(test_email))


Test Email Prediction: Ham
